In [8]:
import requests
import json

# Define the endpoint and payload

myip="http://72.210.56.254:11434"

url = myip+"/api/generate"
payload = {
    "model": "llama3.2",
#    "model": "llama3.3",
#    "model": "qwq",  
#    "model":  "hf.co/bartowski/Llama-3.2-3B-Instruct-GGUF",
    
    "prompt": "math  quandratic formula",
    "stream": False
}
headers = {
    "Content-Type": "application/json"
}

# Send the POST request
try:
    response = requests.post(url, headers=headers, json=payload, timeout=30000)
    response.raise_for_status()  # Raise an HTTPError for bad responses
    print("Response:", response.json())
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


Response: {'model': 'llama3.2', 'created_at': '2024-12-14T14:04:43.9686729Z', 'response': "The quadratic formula is a mathematical formula that provides the solutions to a quadratic equation of the form ax^2 + bx + c = 0.\n\nThe quadratic formula is:\n\nx = (-b ± √(b^2 - 4ac)) / 2a\n\nWhere:\n\n* x is the variable\n* a, b, and c are coefficients of the quadratic equation\n* ± (plus-minus sign) indicates that there may be two solutions\n\nHere's how to use the formula:\n\n1. Identify the values of a, b, and c from the quadratic equation.\n2. Plug these values into the formula: x = (-b ± √(b^2 - 4ac)) / 2a\n3. Simplify the expression under the square root (b^2 - 4ac).\n4. Take the square root of the result.\n5. Divide the result by 2a.\n\nExample:\n\nx = (-3 ± √(9 - 12)) / 2\n\nStep 1: Identify a, b, and c\na = 2, b = -3, c = -6\n\nStep 2: Plug values into formula\nx = (-(-3) ± √((-3)^2 - 4*2*(-6))) / 2\n\nStep 3: Simplify expression under square root\nx = (3 ± √(9 + 48)) / 2\nx = (3 ± √

In [4]:
import requests
import json

# Class to interact with the Llama model server
class LlamaLLM:
    def __init__(self, url):
        self.url = url

    def generate(self, prompt):
        payload = {
            "model": "llama3.2",
            "prompt": prompt,
            "stream": False
        }
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.post(self.url, headers=headers, data=json.dumps(payload),timeout=30000)
        response.raise_for_status()
        return response.json()['response']

# Environment class remains the same
class Environment:
    def __init__(self):
        self.state = 'initial_state'  # Initial state
        self.goal_achieved = False  # Goal not achieved
        self.result = None  # Store result

    def get_state(self):
        return self.state  # Get current state

    def change_state(self, result):
        self.state = 'goal_state'  # Change to goal state
        self.result = result  # Store result

# ReActAgent class modified to use LlamaLLM for reasoning
class ReActAgent:
    def __init__(self, environment, llm, max_attempts=5):
        self.environment = environment  # Initialize environment
        self.llm = llm  # Llama 3.1 model
        self.max_attempts = max_attempts  # Max attempts

    def perceive(self):
        return self.environment.get_state()  # Perceive environment state

    def reason(self, state):
        # Causal reasoning prompt based on paper example
        prompt = """
        Given the following causal relationships:
        - A causes B
        - B causes C
        - A is independent of C given B
        
        Question: Does A directly affect C?
        """
        response = self.llm.generate(prompt).strip()
        print(f"Raw Llama Response: {response}")  # Debugging: print raw response
        if 'No' in response or 'independent' in response:
            return 'achieve_goal', "A does not directly affect C"
        return 'take_action', None

    def act(self, action, result):
        if action == 'achieve_goal':
            self.environment.goal_achieved = True  # Achieve goal
            self.environment.change_state(result)  # Change state and store result

# Initialize the Llama model with the server URL

myip="http://72.210.56.254:11434"

url = myip+"/api/generate"
llm = LlamaLLM(url="http://72.210.56.254:11434/api/generate")

# Create environment and agent
env = Environment()
agent = ReActAgent(env, llm)

# Agent perception, reasoning, and action loop
attempts = 0
while not env.goal_achieved and attempts < agent.max_attempts:
    state = agent.perceive()
    action, result = agent.reason(state)
    agent.act(action, result)
    attempts += 1
    print(f"State: {state}, Action: {action}, Result: {result}, Attempts: {attempts}")

if env.goal_achieved:
    print(f"Goal achieved with result: {env.result}")
else:
    print("Failed to achieve goal within the maximum number of attempts.")

Raw Llama Response: No, based on the information provided:

- A causes B
- B causes C
- A is independent of C given B (meaning if we know that B occurs, then A and C are not related)

This implies that A can cause an effect on C only through its indirect influence on B. In other words, there is no direct causal relationship between A and C.

If you want to determine the strength or likelihood of the relationship between A and C (and thus whether A directly affects C), additional information would be needed.
State: initial_state, Action: achieve_goal, Result: A does not directly affect C, Attempts: 1
Goal achieved with result: A does not directly affect C
